In [1]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
# 載入原始 Keras 模型 (float32)
model = tf.keras.models.load_model('1.3MobileNetV2.h5')

In [4]:
def representative_dataset_gen():
    dataset_dir = 'Garbage'  # 請替換為你的資料夾路徑
    datagen = ImageDataGenerator(rescale=1./255)
    calib_generator = datagen.flow_from_directory(
        dataset_dir,
        target_size=(224, 224),
        batch_size=1,
        class_mode=None,
        shuffle=True
    )

    for i in range(8000):
        img = next(calib_generator)
        yield [img]

In [5]:
# 建立 TFLiteConverter
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# 開啟優化
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_FLOAT16]

In [6]:
# 指定量化方式
converter.target_spec.supported_types = [tf.float16]  # float16 或 int8

In [7]:
# 轉換
tflite_quant_model = converter.convert()

# 儲存 .tflite
with open('1.3MobileNetV2.tflite', 'wb') as f:
    f.write(tflite_quant_model)

INFO:tensorflow:Assets written to: C:\Users\willy\AppData\Local\Temp\tmpy25drazr\assets


c:\Users\willy\anaconda3\envs\tensorflow_gpu\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [8]:
import tensorflow as tf
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, f1_score, confusion_matrix, ConfusionMatrixDisplay

In [9]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [10]:
# === 1. 載入 .tflite 模型 ===
interpreter = tf.lite.Interpreter(model_path="1.3MobileNetV2.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_index = input_details[0]['index']
output_index = output_details[0]['index']

# 輸入尺寸
_, height, width, channels = input_details[0]['shape']

In [11]:
# === 2. 圖片預處理函式 ===

def preprocess_image(img_path):
    img = cv2.imread(img_path)
    if img is None:
        return None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (width, height))
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=0)  # 加入 batch 維度
    return img

In [12]:
# === 3. 讀取測試圖片與標籤 ===
def load_test_data(img_dir):
    X = []
    y = []
    class_names = sorted(os.listdir(img_dir))  # 每個子資料夾為一個類別
    class_to_idx = {name: idx for idx, name in enumerate(class_names)}

    for class_name in class_names:
        class_folder = os.path.join(img_dir, class_name)
        if not os.path.isdir(class_folder):
            continue
        for fname in os.listdir(class_folder):
            if fname.lower().endswith(('.jpg', '.png', '.jpeg')):
                img_path = os.path.join(class_folder, fname)
                img = preprocess_image(img_path)
                if img is not None:
                    X.append(img)
                    y.append(class_to_idx[class_name])
    return X, y, class_names

# 設定測試資料資料夾
test_dir = "trashbox"  # ← 替換為實際測試資料夾路徑
X, y_true, class_names = load_test_data(test_dir)

In [13]:
# === 4. 執行推論 ===
y_pred = []
for img in X:
    interpreter.set_tensor(input_index, img)
    interpreter.invoke()
    output = interpreter.get_tensor(output_index)
    pred = np.argmax(output, axis=-1)[0]
    y_pred.append(pred)

In [14]:
# === 5. 計算 F1-score 與分類報告 ===
accuracy = accuracy_score(y_true, y_pred)
print(f"準確率: {accuracy}")

print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

準確率: 0.719830262885531
              precision    recall  f1-score   support

          BG     0.9325    1.0000    0.9651       373
       Metal     0.4187    0.8707    0.5655      1562
          PG     0.9203    0.5468    0.6860      3696
       Paper     0.8384    0.7941    0.8156      4031

    accuracy                         0.7198      9662
   macro avg     0.7775    0.8029    0.7581      9662
weighted avg     0.8055    0.7198    0.7314      9662

